# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Weights and Biases

In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 1.8MB 8.6MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 133kB 30.4MB/s 
     |████████████████████████████████| 163kB 26.0MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 


# Configure wandb sweep

In [ ]:
from wandb.keras import WandbCallback
import wandb

In [ ]:
# Configure the sweep
sweep_config = {
    'method': 'grid',
    'name' : 'Neuron & LR Sweep - Batch Size 64 (Crashed Runs)',
    'parameters': {
        'learning_rate': {
            'values': [5e-4, 1e-5]
        },
        'fc3_num_neurons': {
            'values': [256, 512]
        },
    }
}

In [ ]:
WANDB_API_KEY='002b3552fd78e3559e38c7317ef0d9eefd0c7c6d' 
sweep_id = wandb.sweep(sweep_config, project="nn_project_tuning")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 30jubtgn
Sweep URL: https://wandb.ai/burntice/nn_project_tuning/sweeps/30jubtgn


# Model

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf

# Load dataset as dataframe
base_path = '/content/drive/My Drive/Y4 Sem 1/NN Project/'
df = pd.read_csv('/content/drive/My Drive/Y4 Sem 1/NN Project/aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_val_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_val_df, test_size=0.2)

# function to call for hyperparameter sweep
def train():
    # Fix seed
    seed = 7
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # default hyperparameters
    defaults = {
        'epochs': 20,
        'batch_size': 64,
        'learning_rate': 1e-5,
        'weight_decay': 0.01,
        'fc1_num_neurons': 512,
        'fc2_num_neurons': 512,
        'fc3_num_neurons': 256,
        'dropout1_rate': 0,
        'dropout2_rate': 0,
        'dropout3_rate': 0,
        'seed': seed,
        'optimizer': 'adam',
        'hidden_activation': 'relu',
        'output_activation': 'sigmoid',
        'loss_function': 'binary_crossentropy',
        'metrics': ['accuracy'],
    }

    wandb.init(
        config=defaults,
        name='Hyperparameter Tuning',
        project='nn_project',
    )
    config = wandb.config

    # Define model
    mobile_net_v2 = tf.keras.applications.MobileNetV2(
        include_top=False,
        pooling='avg',
        weights='imagenet',
        input_shape=(224,224,3),
    )

    fc1 = tf.keras.layers.Dense(
        config.fc1_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc2 = tf.keras.layers.Dense(
        config.fc2_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    fc3 = tf.keras.layers.Dense(
        config.fc3_num_neurons,
        activation=config.hidden_activation,
        kernel_regularizer=tf.keras.regularizers.L2(l2=config.weight_decay),
    )

    bn1 = tf.keras.layers.BatchNormalization()
    bn2 = tf.keras.layers.BatchNormalization()
    bn3 = tf.keras.layers.BatchNormalization()
    bn4 = tf.keras.layers.BatchNormalization()

    dropout1 = tf.keras.layers.Dropout(rate=config.dropout1_rate)
    dropout2 = tf.keras.layers.Dropout(rate=config.dropout2_rate)
    dropout3 = tf.keras.layers.Dropout(rate=config.dropout3_rate)

    model = tf.keras.models.Sequential([
        mobile_net_v2,
        tf.keras.layers.Flatten(),
        bn1,
        fc1,
        dropout1,
        bn2,
        fc2,
        dropout2,
        bn3,
        fc3,
        dropout3,
        bn4,
        tf.keras.layers.Dense(1, activation=config.output_activation),
    ])

    # Compile model 
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
        loss=config.loss_function,
        metrics=config.metrics,
    )

    # Load images into keras image generator 
    datagen_train = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_val = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )
    datagen_test = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    )

    train_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    val_generator = datagen_train.flow_from_dataframe(
        dataframe=val_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    test_generator = datagen_test.flow_from_dataframe(
        dataframe=test_df,
        x_col='datadir',
        y_col='gender',
        batch_size=config.batch_size,
        seed=seed,
        shuffle=True,
        class_mode='raw',
        target_size=(224,224),
    )

    model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=config.epochs,
        callbacks=[WandbCallback()],
    )

# Run agent

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: bqgynndx with config:
wandb: 	fc3_num_neurons: 256
wandb: 	learning_rate: 0.0005
wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


9412608/9406464 [==============================] - 0s 0us/step
Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 2411s 20s/step - loss: 12.1532 - accuracy: 0.8392 - val_loss: 8.2029 - val_accuracy: 0.8375
Epoch 2/20
122/122 [==============================] - 140s 1s/step - loss: 5.3090 - accuracy: 0.9416 - val_loss: 3.9156 - val_accuracy: 0.7125
Epoch 3/20
122/122 [==============================] - 144s 1s/step - loss: 1.9649 - accuracy: 0.9667 - val_loss: 1.8405 - val_accuracy: 0.8216
Epoch 4/20
122/122 [==============================] - 142s 1s/step - loss: 0.6912 - accuracy: 0.9717 - val_loss: 1.1460 - val_accuracy: 0.8278
Epoch 5/20
122/122 [==============================] - 142s 1s/step - loss: 0.2897 - accuracy: 0.9732 - val_loss: 2.4218 - val_accuracy: 0.6269
Epoch 6/20
122/122 [==============================] - 140s 1s/step - loss: 0.1458 - accuracy: 0.9771 - va

epoch,19
loss,0.08207
accuracy,0.97988
val_loss,0.23134
val_accuracy,0.92209
_step,19
_runtime,5186
_timestamp,1605986405
best_val_loss,0.22994
best_epoch,12


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▆▇▇▇█████▇█████████
val_loss,█▄▂▂▃▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁
val_accuracy,▆▃▆▆▁▃█▇▇▆███▅█▇▆▅▆█
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▇▇▇██


wandb: Agent Starting Run: zrxfvmfo with config:
wandb: 	fc3_num_neurons: 256
wandb: 	learning_rate: 1e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Found 7804 validated image filenames.
Found 1951 validated image filenames.
Found 2439 validated image filenames.
Epoch 1/20
122/122 [==============================] - 141s 1s/step - loss: 16.5035 - accuracy: 0.6046 - val_loss: 16.3446 - val_accuracy: 0.6202
Epoch 2/20
122/122 [==============================] - 141s 1s/step - loss: 16.1200 - accuracy: 0.7560 - val_loss: 16.1336 - val_accuracy: 0.6786
Epoch 3/20
122/122 [==============================] - 142s 1s/step - loss: 15.8362 - accuracy: 0.8298 - val_loss: 15.9322 - val_accuracy: 0.6986
Epoch 4/20
 25/122 [=====>........................] - ETA: 1:29 - loss: 15.6694 - accuracy: 0.8672